# Model

> Dev notebook

In [1]:
# | default_exp training

In [2]:
# | hide
%reload_ext autoreload
%reload_ext nb_black
%autoreload 2
from nbdev.showdoc import *
import sys

__root = "../"
sys.path.append(__root)


<IPython.core.display.Javascript object>

In [3]:
import itertools
from torch_snippets import *
from clip.core import download_flickr8k_from_kaggle
from clip.config import ClipConfig
from clip.dataset import CLIPDataset
from clip.models import CLIP

<IPython.core.display.Javascript object>

In [4]:
# | master
kaggle_json_path = P("/home/yyr/Downloads/kaggle.json")
data_download_path = P("/home/yyr/Zapdos/ml-datasets/flickr-8k-kaggle/")
download_flickr8k_from_kaggle(kaggle_json_path, data_download_path)

df = pd.read_csv(data_download_path / "captions.txt")
df["id"] = [id_ for id_ in range(len(df) // 5) for _ in range(5)]
df.to_csv(data_download_path / "captions.csv")

[09/23/23 21:11:16] INFO     Loaded key from /home/yyr/Downloads/kaggle.json                                                                      ]8;id=334599;file:///mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/../clip/core.py:15\core.py]8;;\:]8;id=224002;file:///mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/../clip/core.py:15#load_kaggle_key:15\load_kaggle_key:15]8;;\

                    INFO     Data is already downloaded at /home/yyr/Zapdos/ml-datasets/flickr-8k-kaggle                            ]8;id=411276;file:///mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/../clip/core.py:24\core.py]8;;\:]8;id=751506;file:///mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/../clip/core.py:24#download_flickr8k_from_kaggle:24\download_flickr8k_from_kaggle:24]8;;\

<IPython.core.display.Javascript object>

In [5]:
config = ClipConfig()
config.image_path = data_download_path / "Images"
config.captions_csv_path = data_download_path / "captions.csv"
config.debug = False  # Switch to True case you want to reduce the dataset size

<IPython.core.display.Javascript object>

In [6]:
# trn_dl, val_dl = build_clip_data_loaders(config)
trn_ds, val_ds = CLIPDataset.train_test_split(config)

model = CLIP(config).to(config.device)
params = [
    {"params": model.image_encoder.parameters(), "lr": config.image_encoder_lr},
    {"params": model.text_encoder.parameters(), "lr": config.text_encoder_lr},
    {
        "params": itertools.chain(
            model.image_projection.parameters(), model.text_projection.parameters()
        ),
        "lr": config.head_lr,
        "weight_decay": config.weight_decay,
    },
]
optimizer = torch.optim.AdamW(params, weight_decay=0.0)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=config.patience, factor=config.factor
)


Output()

[09/23/23 21:11:47] INFO     Creating encoded captions for train dataset... - Completed in 28.63 s                                                ]8;id=455140;file:///home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/contextlib.py:120\contextlib.py]8;;\:]8;id=136547;file:///home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/contextlib.py:120#__exit__:120\__exit__:120]8;;\

Output()

[09/23/23 21:11:57] INFO     Creating encoded captions for valid dataset... - Completed in 8.96 s                                                 ]8;id=272822;file:///home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/contextlib.py:120\contextlib.py]8;;\:]8;id=78757;file:///home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/contextlib.py:120#__exit__:120\__exit__:120]8;;\

<IPython.core.display.Javascript object>

In [7]:
from transformers import Trainer, TrainingArguments

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory where checkpoints and logs will be saved.
    num_train_epochs=config.epochs,  # Total number of training epochs.
    per_device_train_batch_size=config.batch_size,  # Batch size per GPU.
    per_device_eval_batch_size=config.batch_size,  # Batch size for evaluation per GPU.
    evaluation_strategy="steps",  # Evaluation strategy (steps, epoch).
    logging_strategy="steps",  # Logging strategy (steps, epoch).
    save_strategy="steps",  # Save strategy (steps, epoch).
    save_total_limit=2,  # Limit the total amount of checkpoints.
    learning_rate=5e-5,  # Learning rate.
    logging_steps=config.save_eval_and_logging_steps,
    save_steps=config.save_eval_and_logging_steps,  # Save checkpoints every N steps.
    eval_steps=config.save_eval_and_logging_steps,  # Evaluate every N steps.
    logging_dir="./logs",  # Directory for storing logs.
    metric_for_best_model="loss",
    label_names=["image", "input_ids"],
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trn_ds,
    eval_dataset=val_ds,
    optimizers=(optimizer, lr_scheduler),  # Pass your custom optimizer here.
)
# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

Step,Training Loss,Validation Loss
500,2.898400,2.246151
1000,1.095200,2.187405
1500,0.677100,2.177920
2000,0.538000,2.218436
2500,0.388600,2.283489
3000,0.321500,2.317244
3500,0.243800,2.431228
4000,0.221200,2.317610


{'eval_loss': 2.3055901527404785,
 'eval_runtime': 83.5751,
 'eval_samples_per_second': 96.799,
 'eval_steps_per_second': 3.027,
 'epoch': 4.0}

<IPython.core.display.Javascript object>

In [8]:
# | hide

import nbdev
nbdev.nbdev_export()
import subprocess

subprocess.run(["/home/yyr/anaconda3/envs/mcvp-book/bin/black", __root])


/home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/site-packages/nbdev/export.py:54: UserWarning: Notebook '/mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/02.00_training.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/core.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/config.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/_modidx.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/dataset.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/models.py

All d

CompletedProcess(args=['/home/yyr/anaconda3/envs/mcvp-book/bin/black', '../'], returncode=0)

<IPython.core.display.Javascript object>